# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [302]:
# Import necesary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [310]:
# Define the function scrape books with the arguments min_rating and max_price
def scrape_books(min_rating, max_price):
        
    # Define functions to get title, url and price
    def get_title(book):
        title = book.find_all("a")[1]["title"]
        return title
    def get_url(book):
        book_url = book.find_all("a")[1]["href"]
        domain = "https://books.toscrape.com/catalogue/"
        return domain+book_url
    def get_price(book):
        price = book.find("p", attrs = {"class":"price_color"}).get_text().replace('£','')
        return price
    def get_availability(book):
        availability = book.find("p", attrs = {"class":"instock availability"}).get_text().replace('\n', '')
        return availability
    
    # Define functions to get upc, genre, rating and description
    # Atention! These are to be performed in the book's soup, not the general one!
    def get_upc(book):
        upc = book.find("td").get_text()
        return upc
    def get_genre(book):
        genre = book.find("ul", attrs={'class':'breadcrumb'}).find_all('li')[2].get_text().strip()
        return genre
    def get_rating(book):
        rating = book.find_all('p')[2].get('class')[1].lower()
        my_nums = {'one': 1,
                   'two': 2,
                   'three': 3,
                   'four': 4,
                   'five': 5,
        }
        return my_nums[rating]
    def get_description(book):
        description = book.find_all('p')[3].get_text()
        return description
    
    # Create an empty dictionary for books and initialise key
    books_dict = {}
    key = 0
    
    # Get grids for all 50 pages
    for number in range(1,51):
        # Create a request to access the HTML code of each page
        url = f"https://books.toscrape.com/catalogue/page-{str(number)}.html"
        response = requests.get(url)
        # Create a soup object from the HTML code 
        soup = BeautifulSoup(response.content, 'html.parser')
    
        # Find all the books in a given page
        book_grid = soup.find("ol", attrs = {"class":"row"})
        # Inside of the grid identify all books
        books = book_grid.find_all("li", attrs = {"class":"col-xs-6 col-sm-4 col-md-3 col-lg-3"})
    
        # Iterate functions to all books 
        for book in books:
            title = get_title(book)
            price = get_price(book)
            availability = get_availability(book)
            book_url = get_url(book)
    
            ## Access the book url to retrieve more data.
            book_response = requests.get(book_url)
            ## 1st generate a soup for the book
            book_soup = BeautifulSoup(book_response.content, 'html.parser')
    
            ## Start gathering info from the book's page
            upc = get_upc(book_soup)
            genre = get_genre(book_soup)
            description = get_description(book_soup)
            rating = get_rating(book_soup)
    
            #Finally, save the retrieved data in a dictionary
            books_dict[key] = {"title": title,
                               "genre": genre,
                               "price_£": price,
                               "availability": availability,
                               "rating": rating,
                               "upc": upc,
                               "description": description,
          }
            key += 1
    # Turn the dictionary into a dataframe    
    books_df = pd.DataFrame.from_dict(books_dict, orient = "index")
    books_df['price_£'] = books_df['price_£'].astype('float32')
    condition = (books_df['rating'] >= min_rating) & (books_df['price_£'] <= max_price)
    return books_df[condition]

In [309]:
scrape_books(4,20)

,title,genre,price_£,availability,rating,upc,description
12,Set Me Free,Young Adult,17.459999,In stock,5,ce6396b0f23f6ecc,Aaron Ledbetter’s future had been planned out ...
